In [1]:
from image_generation import generate_single_example
from utils import return_base_image_paths, generate_categories
from pprint import pprint
from tqdm import tqdm
import psycopg2 as psql

In [2]:
try:
    conn = psql.connect(
        host='localhost',
        user='postgres',
        password='newpassword',
        port=5432,
        database='postgres'
    )
    
    c = conn.cursor()
finally:
    conn.close()

In [3]:
img_files = return_base_image_paths()
TEXT = [
    "Name: ",
    "Address: ",
    "Beneficiary: ",
    "Phone Number: ",
    "Age: "
]

Number of Base Word Directories: 76


In [4]:
data = generate_single_example(
    TEXT,
    img_files,
    "write_test",
    print_img=True
)

pprint(data)

No Database Connection
{'annotations': [{'area': 6981,
                  'bbox': (104, 206, 179, 39),
                  'category_id': 2,
                  'id': 'e0ec057b-dd94-4e20-9b34-cc9d9a588ff0',
                  'image_id': '30a66fb8-9cd3-4c46-b4c8-f66a1497906e',
                  'iscrowd': 0},
                 {'area': 7139,
                  'bbox': (434, 188, 121, 59),
                  'category_id': 3,
                  'id': '9154d7ad-357e-4666-86ff-3aca0fe2a5af',
                  'image_id': '30a66fb8-9cd3-4c46-b4c8-f66a1497906e',
                  'iscrowd': 0},
                 {'area': 1316,
                  'bbox': (330, 244, 188, 7),
                  'category_id': 1,
                  'id': 'ebf84424-d6d4-4bc1-bbf0-26eca2d64313',
                  'image_id': '30a66fb8-9cd3-4c46-b4c8-f66a1497906e',
                  'iscrowd': 0}],
 'image': {'filename': '30a66fb8-9cd3-4c46-b4c8-f66a1497906e.png',
           'height': 400,
           'id': '30a66fb8-9cd3-4c46-b

In [5]:
DATA = []

try:
    conn = psql.connect(
        host='localhost',
        user='postgres',
        password='newpassword',
        port=5432,
        database='postgres'
    )
    
    for _ in tqdm(range(500)):
        Data = generate_single_example(
            TEXT,
            img_files,
            "write_test",
            print_img=True,
            db_conn=conn
        )
        DATA.append(Data)
finally:
    conn.close()

100%|██████████| 500/500 [00:09<00:00, 53.88it/s]


In [6]:
annotations = []
images = []

for d in DATA:
    annotations.extend(d['annotations'])
    images.append(d['image'])

In [7]:
JSON_DATA = {
    "annotations": annotations,
    "images": images,
    "categories": generate_categories(),
    "info": {},
    "licenses": {}
}

In [8]:
def send_file_query(cur,
                    conn,
                    fpath: str,
                    height: int,
                    width: int,
                    file_encoding: str='png',
                    ):
    """
    
    """
    query = f"""
    INSERT INTO handwriting_files (fpath, height, width, file_encoding)
    VALUES 
    (
        {fpath},
        {height},
        {width},
        {file_encoding}
    );
    """
    success = True
    try:
        cur.execute(query)
    except Exception as e:
        print(f"Exception: {e}")
        success = False
    finally:
        conn.commit()
    return success

In [9]:
def send_bbox_query():
    """
    
    """
    query = f"""
    INSERT INTO handwriting_bboxes (class_text, class_id, x, y, w, h, ref_image, text)
    VALUES
    (
        {class_text},
        {class_id},
        {x},
        {y},
        {w},
        {h},
        {ref_image},
        {text}
    );
    """
    success = True
    try:
        cur.execute(query)
    except Exception as e:
        print(f"Exception: {e}")
        success = False
    finally:
        conn.commit()
    return success